In [ ]:
%load_ext autoreload
%autoreload 2

In [14]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt

import numpy as np
import pandas as pd; pd.options.mode.chained_assignment = None

import APIs.Bloomberg as ba

import Utilities.Weather as uw
import Utilities.GLOBAL as GV
import plotly.express as px

In [9]:
blp = ba.BLPInterface('//blp/exrsvc')

In [12]:
location = 'US_IL'
forecast = 'GFS'

overrides = {'location': location, 'fields':'TEMPERATURE|PRECIPITATION', 'model':forecast,'publication_date':'2022-06-29T00:00:00'}

df_GFS = blp.bsrch('comdty:weather', overrides)
df_GFS

,Reported Time,Temperature (°C),Precipitation (mm)
0,2022-06-29 00:00:00+00:00,26.64,
1,2022-06-29 03:00:00+00:00,21.77,0.15
2,2022-06-29 06:00:00+00:00,18.97,0.85
3,2022-06-29 09:00:00+00:00,17.83,0.48
4,2022-06-29 12:00:00+00:00,18.60,0.19
...,...,...,...
124,2022-07-14 12:00:00+00:00,26.43,0.12
125,2022-07-14 15:00:00+00:00,31.43,0.01
126,2022-07-14 18:00:00+00:00,34.30,0.0
127,2022-07-14 21:00:00+00:00,35.65,0.01
